<a href="https://colab.research.google.com/github/ssifood/CoLA_backend/blob/main/lxper_mission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

머신러닝 nlp엔지니어 과제
huggingface transformers 기반 모델과 fastapi 라이브러리로 다음의 기능 수행하는 단순 api 웹서버 구현

a.요건

    GLUE task중 CoLA 데이터를 활용해, 주어진 문장이 문법적으로 적합한지를 판별하는 API를 개발
    -최소한 3개의 huggingface transformers에서 제공하는 prtrained model에 GLU CoLa데이터셋을 입력해 finetunning하고, 이중 BEST모형을 선택한다.
    -선택된 모형을 fastapi 라이브러리를 활용해 서비스하고,서비스 포트는 8000번을 활용
    -/generate 엔드포인트는 json payload를 HTTP POST 방식으로 이벽받아 JSON response를 반환하여야 한다.

        -입력값:{"passage":"I are a boy"}
            -passage라는 키값을 지니며, 이 키의 값은 문자열로, 문법적합성을 판정할 문장이다.
        -출력값: {"prob":0.001}
            -prob라는 키를 가지며, 이 키의 값은 실수로 ,문법적합성을  나타내는 값이다. 1에 가까울 수록 문법적으로 적합함을 나타낸다.

        

In [5]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 53780, done.
remote: Total 53780 (delta 0), reused 0 (delta 0), pack-reused 53780
Receiving objects: 100% (53780/53780), 40.23 MiB | 30.29 MiB/s, done.
Resolving deltas: 100% (37521/37521), done.


In [2]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 1.3MB 24.1MB/s 
     |████████████████████████████████| 1.1MB 50.8MB/s 
     |████████████████████████████████| 2.9MB 58.8MB/s 
     |████████████████████████████████| 890kB 53.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=3a8d5b52b9fd0175b09d24e53ef597bfdb5809de14ce68156b230932af794c05
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 163kB 14.6MB/s 
     |████████████████████████████████| 17.7MB 197kB/s 
     |████████████████████████████████| 245kB 55.6MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [6]:
from datasets import load_dataset
dataset = load_dataset('glue','cola')

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/cola/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)


In [7]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})


In [7]:
import os
os.chdir("/content/transformers/examples/text-classification")
!ls

README.md    run_pl_glue.py  run_tf_glue.py		    run_xnli.py
run_glue.py  run_pl.sh	     run_tf_text_classification.py


In [8]:
!export TASK_NAME=COLA

In [9]:
!echo $TASK_NAME
!echo TASK_NAME
#이거왜 리눅스 전역변수가안되냐?? 그래서 $TASK_NAME 수동으로 입력


TASK_NAME


In [10]:
!python run_glue.py \
  --model_name_or_path bert-base-cased \
  --task_name cola \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir /tmp/cola/

2020-11-30 00:59:49.869873: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/30/2020 00:59:51 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
11/30/2020 00:59:51 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/tmp/cola/', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov30_00-59-51_c72c417f1f67', logging_first_st

In [ ]:
!python run_glue.py \
  --model_name_or_path xlm-roberta-base \
  --task_name cola \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3.0 \
  --output_dir /tmp/cola_xlm-roberta-base/

2020-11-30 01:20:40.216112: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/30/2020 01:20:41 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
11/30/2020 01:20:41 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/tmp/cola_xlm-roberta-base/', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov30_01-20-41_c72c417f1f67',